In [1]:
import traceback

import polars as pl
import sys
import torch
import polars as pl
import numpy as np
from sklearn.linear_model import LinearRegression
from lifelines import WeibullFitter
from sklearn.model_selection import train_test_split
from torch import optim
import torch
import torch.nn as nn

from torch.utils.data import Dataset, DataLoader

from models.Titans import Model
from utils.custom_losses import CustomLoss
from utils.lstf_feature_maker.piecewise_linear_regression import PiecewiseLinearRegression
from utils.lstf_feature_maker.weibull import WeibullFeatureMaker

'''
pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu128
https://developer.nvidia.com/cuda-12-8-0-download-archive
'''

MAC_DIR = '../data/'
WINDOW_DIR = 'C:/Users/USER/PycharmProjects/research/data/'

if sys.platform == 'win32':
    DIR = WINDOW_DIR
    print(torch.cuda.is_available())
    print(torch.cuda.device_count())
    print(torch.version.cuda)
    print(torch.__version__)
    print(torch.cuda.get_device_name(0))
    print(torch.__version__)
else:
    DIR = MAC_DIR

tb_bas_oper_part_mst = (pl.read_parquet(DIR + 'tb_bas_oper_part_mst.parquet')
                        .select(['OPER_PART_NO', 'OPER_PART_NM'])
                        .rename({'OPER_PART_NO': 'oper_part_no', 'OPER_PART_NM': 'oper_part_nm'}))
tb_dyn_fcst_demand = (pl.read_parquet(DIR + 'tb_dyn_fcst_dmnd.parquet')
                      .select(['PART_NO', 'DMND_QTY', 'DMND_DT', 'OPER_PART_NO'])
                      .rename({'PART_NO': 'part_no', 'OPER_PART_NO': 'oper_part_no', 'DMND_DT': 'demand_dt', 'DMND_QTY': 'demand_qty'})
                      .select(['part_no', 'oper_part_no', 'demand_dt', 'demand_qty']))
tb_dyn_fcst_demand_sellout = (pl.read_parquet(DIR + 'tb_dyn_fcst_dmnd_sellout.parquet')
                              .select(['PART_NO', 'DMND_QTY', 'DMND_DT', 'OPER_PART_NO'])
                              .rename({'PART_NO': 'part_no', 'OPER_PART_NO': 'oper_part_no', 'DMND_DT': 'demand_dt', 'DMND_QTY': 'demand_qty'})
                              .select(['part_no', 'oper_part_no', 'demand_dt', 'demand_qty']))

True
1
12.8
2.9.0.dev20250716+cu128
NVIDIA GeForce RTX 5080
2.9.0.dev20250716+cu128


In [3]:
from utils.date_util import DateUtil

# dyn_fcst_demand = tb_dyn_fcst_demand.with_columns([
#     pl.col('demand_dt').cast(pl.Int64).map_elements(DateUtil.yyyymmdd_to_date, return_dtype = pl.Date)
# ])

dyn_demand_sellout = tb_dyn_fcst_demand_sellout.with_columns([
    pl.col('demand_dt').cast(pl.Int64).map_elements(DateUtil.yyyymmdd_to_date, return_dtype = pl.Date)
])


### Monthly Preprocessing

In [7]:
dyn_demand_monthly = (dyn_demand_sellout.join(tb_bas_oper_part_mst, on = 'oper_part_no', how = 'left')
                    .select(['oper_part_no', 'oper_part_nm', 'demand_dt', 'demand_qty'])
                    .sort(['oper_part_no', 'demand_dt'])
                    .with_columns([
                        pl.col('demand_qty').cum_sum().over('oper_part_no').alias('cumsum_qty')
                 ])
               )
dyn_demand_monthly = (dyn_demand_monthly
        .select(['oper_part_no', 'demand_dt', 'demand_qty', 'cumsum_qty'])
        .sort(['oper_part_no', 'demand_dt'])
        .with_columns(pl.col('demand_dt').map_elements(DateUtil.date_to_yyyymm, return_dtype = pl.Int64).alias('demand_dt'))
        .select('oper_part_no', 'demand_dt', 'demand_qty')
        .group_by(['oper_part_no', 'demand_dt'])
        .agg(pl.col('demand_qty').sum().alias('demand_qty'))
        .sort(['oper_part_no', 'demand_dt'])
        # .write_parquet(DIR + 'target_dyn_demand_monthly.parquet')
 )

In [27]:
dyn_demand_monthly.filter(pl.col('oper_part_no').is_in(['01070-51445', '01070-51470']))

oper_part_no,demand_dt,demand_qty
str,i64,f64
"""01070-51445""",201801,1328.0
"""01070-51445""",201803,460.0
"""01070-51445""",201804,25.0
"""01070-51445""",201805,530.0
"""01070-51445""",201806,470.0
…,…,…
"""01070-51470""",202607,25.0
"""01070-51470""",202609,15.0
"""01070-51470""",202611,19.0


### Weekly Preprocessing

In [5]:
(dyn_demand_sellout
      .join(tb_bas_oper_part_mst, on = 'oper_part_no', how = 'left')
      .select(['oper_part_no', 'oper_part_nm', 'demand_dt', 'demand_qty'])
      .sort(['oper_part_no', 'demand_dt'])
      .with_columns([
        pl.col('demand_qty').cum_sum().over('oper_part_no').alias('cumsum_qty')
      ])
      .select(['oper_part_no', 'demand_dt', 'demand_qty', 'cumsum_qty'])
      .sort(['oper_part_no', 'demand_dt'])
      .with_columns(pl.col('demand_dt').map_elements(DateUtil.date_to_yyyyww, return_dtype = pl.Int64).alias('demand_dt'))
      .select('oper_part_no', 'demand_dt', 'demand_qty')
      .group_by(['oper_part_no', 'demand_dt'])
      .agg(pl.col('demand_qty').sum().alias('demand_qty'))
      .sort(['oper_part_no', 'demand_dt'])
      .write_parquet(DIR + 'target_dyn_demand_weekly.parquet')
 )